In [ ]:
pc_name = "BrunoMacM1Air"

In [ ]:
import os
import yaml

import numpy as np
from nanopyx.core.transform import NLMDenoising

def concatenate_benchmarks(results, args_repr, previous_dict):
    if args_repr not in previous_dict:
        previous_dict[args_repr] = {}
        
    for t2run,name,_ in results:
        if name in previous_dict[args_repr]:
            previous_dict[args_repr][name].append(t2run)
        else:
            previous_dict[args_repr][name] = [t2run]
    
    return previous_dict


denoising = NLMDenoising(verbose=False,testing=True)
denoising._run_types.pop('threaded_dynamic')
denoising._run_types.pop('threaded_static')
denoising._run_types.pop('threaded_guided')
denoising._run_types.pop('python')

denoising_results = {}

n_repeats = 5

In [ ]:
nlm_image_1 = np.random.random((1, 1000, 1000)).astype(np.float32)
nlm_image_2 = np.random.random((1, 500, 500)).astype(np.float32)
nlm_image_3 = np.random.random((1, 200, 200)).astype(np.float32)

args_1 = [nlm_image_1]
kwargs_1 = {'patch_size': 50, 'patch_distance': 50, 'h': 0.1, 'sigma': 1.0}
args_repr_1,_ = conv._get_args_repr_score(*args_1, **kwargs_1)

args_2 = [nlm_image_2]
kwargs_2 = {'patch_size': 5, 'patch_distance': 100, 'h': 0.1, 'sigma': 1.0}
args_repr_2,_ = conv._get_args_repr_score(*args_2, **kwargs_2)

args_3 = [nlm_image_3]
kwargs_3 = {'patch_size': 5, 'patch_distance': 10, 'h': 0.1, 'sigma': 1.0}
args_repr_3,_ = conv._get_args_repr_score(*args_3, **kwargs_3)

In [ ]:
for i in range(n_repeats): 
    results = denoising.benchmark(*args_1, **kwargs_1)
    denoising_results = concatenate_benchmarks(results, args_repr_1, denoising_results)
    results = denoising.benchmark(*args_2, **kwargs_2)
    denoising_results = concatenate_benchmarks(results, args_repr_2, denoising_results)
    results = denoising.benchmark(*args_3, **kwargs_3)
    denoising_results = concatenate_benchmarks(results, args_repr_3, denoising_results)


In [ ]:
if not os.path.exists("Benchmark_results"):
    os.mkdir("Benchmark_results")

with open(f"Benchmark_results/{pc_name}_Results_denoising.yaml", 'w') as f:
    yaml.dump(denoising_results, f)